In [1]:
# import modules
from post_proc import _filter as filter
from post_proc import _mask as mc
from post_proc import _resample as rsam
from post_proc import _reclass as rc
from post_proc import _add_meta as addm
from post_proc import _extend_water as extdw
from post_proc import _water_body as wtrb

# import libraries
import boto3
import  rasterio 
from rasterio.mask import mask
from rasterio.io import MemoryFile
from rasterio.transform import from_origin
from rasterio.warp import reproject, calculate_default_transform, CRS, Resampling
from shapely.geometry import box
from shapely.geometry import mapping
from skimage.morphology import disk
from skimage.filters import rank
import numpy as np
import io
import geopandas as gpd

## Reading data from S3

In [ ]:
aws_access_key_id = 'Your_aws_access_key_id'
aws_secret_access_key = 'Your_aws_secret_access_key'

In [ ]:
"""Accessing the S3 buckets using boto3 client"""
s3_client = boto3.client('s3')
s3_bucket_name='Your_s3_bucket_name'
s3 = boto3.resource('s3',
                    aws_access_key_id= aws_access_key_id,
                    aws_secret_access_key=aws_secret_access_key,
                    region_name = 'Your_region_name')

In [ ]:
""" Getting data files from the AWS S3 bucket as denoted above and printing the first 10 file names having prefix "2019/7/8" """
my_bucket = s3.Bucket(s3_bucket_name)
bucket_list = []
for file in my_bucket.objects.filter(Prefix = 'output/segmentation/2019/m_3810645_ne_13_060_20190922.tif'):
    file_name = file.key
    if file_name.find(".tif") != -1:
        bucket_list.append(file.key)
length_bucket_list = print(len(bucket_list))

In [ ]:
# Run for all tiles
for file in bucket_list:
    obj = s3.Object(s3_bucket_name,file)
    data = obj.get()['Body'].read()
    clasified_src = rasterio.open(io.BytesIO(data)) 
    clasified_array = clasified_src.read(1)
    clasified_array_meta = clasified_src.meta
    clasified_transform = clasified_src.transform
    clasified_bounds = clasified_src.bounds

    # Clip Back tiles
    shapefile_path = '/mnt/c/Lynker/Beaver_project/naip_shape/NAIP_19_CO_UTM13_reduced.shp' # Path to the tiles shapefile
    # Specify the attribute value to use for clipping
    target_attribute_value = file[25:37]
    # Read the shapefile
    gdf = gpd.read_file(shapefile_path)
    # Find the feature that matches the target attribute value
    matching_feature = gdf[gdf['tile'] == target_attribute_value].iloc[0]
    geometry = matching_feature.geometry
    coords = matching_feature.geometry.bounds
    bounding_box = rasterio.coords.BoundingBox(coords[0], coords[1], coords[2], coords[3])
    
    # Mask the raster using the geometry
    clipped_array, clipped_transform = mask(clasified_src, [mapping(geometry)], crop=True)
    
    # Define parameters for the in-memory dataset
    channel, height, width = clipped_array.shape
    clipped_array = clipped_array[0,:,:]
    dtype = clipped_array.dtype
    crs = rasterio.crs.CRS.from_epsg(26913)  # Example CRS (WGS84)
    # Create an in-memory file-like object
    memfile = MemoryFile()
    # Write the array data to the in-memory dataset
    with memfile.open(
        driver='GTiff',
        height=height,
        width=width,
        count=1,  # Number of bands
        dtype=dtype,
        crs=crs,
        bounds= bounding_box,
        transform = clipped_transform
    ) as dst:
        dst.write(clipped_array, 1)  # Write the array to band 1
    # Open the in-memory dataset as a DatasetReader (src) object
    clipped_src = memfile.open()
    # Now you can work with the src object, for example, you can access its metadata
    clipped_meta = clipped_src.meta
    clipped__transform = clipped_src.transform
    clipped_bounds = clipped_src.bounds
    ## Make geometry
    mask_geometry = box(*clipped_bounds)
    mask_geometry_gpd = gpd.GeoDataFrame( geometry= [mask_geometry])
    
    # Join up the waterways
    final_array = extdw.joinWtr(clipped_array)
    
    # Fill in lakes with holes
    seg_array = wtrb.waterBody(final_array)
    
    # Crop and resample
    
    # LCMAP mask (urban mask)
    LCMAP_crop = mc.cropMask('/mnt/c/Lynker/Beaver_project/LCMAP/new_gdal_lcmap.tif', mask_geometry_gpd, clipped_meta)  
    LCMAP_resam = rsam.resampleRaster(LCMAP_crop, clipped_src)  
    
    # Agg mask
    cdl_crop = mc.cropMask('/mnt/c/Lynker/Beaver_project/Downloaded_images/agg_final.tif', mask_geometry_gpd, clipped_meta)
    cdl_resam = rsam.resampleRaster(cdl_crop, clipped_src)   
    del LCMAP_crop, cdl_crop
    
    # Reclassification of all groups- We are not masking class 1 and 2 
    # Water reclassification
    reclass_water = rc.waterReclass(clipped_array, clipped_meta) 
    with rasterio.open(reclass_water.name) as water_src:
        reclass_water = water_src.read(1)
    
    # Aquatic bed reclassification
    reclass_aqua = rc.aquaReclass(clipped_array, clipped_meta)  
    with rasterio.open(reclass_aqua.name) as aquatic_src:
        reclass_aqua = aquatic_src.read(1)
    
    # Emergent reclassification
    reclass_emerg = rc.emergReclass(clipped_array, clipped_meta) 
    reclass_emerg = filter.classesFilter(reclass_emerg, LCMAP_resam, cdl_resam) 
    
    # Scrub reclassification
    reclass_scrub = rc.scrubReclass(clipped_array, clipped_meta)
    reclass_scrub = filter.classesFilter(reclass_scrub, LCMAP_resam, cdl_resam) 
    # Forest reclassification
    reclass_forest = rc.forestReclass(clipped_array, clipped_meta) 
    reclass_forest = filter.classesFilter(reclass_forest, LCMAP_resam, cdl_resam) 
    
    # Shoreline reclassification 
    reclass_shore = rc.shoreReclass(clipped_array, clipped_meta) 
    reclass_shore = filter.classesFilter(reclass_shore, LCMAP_resam, cdl_resam) 
    
    # Classify from 2-6 classes
    reclass_aqua1 = reclass_aqua*2
    reclass_emerg1 = reclass_emerg*3
    reclass_scrub1 = reclass_scrub*4
    reclass_forest1 = reclass_forest*5
    reclass_shore1 = reclass_shore*6
    group1_2 = np.sum([reclass_water, reclass_aqua1], axis = 0)  # water and aquatic bed classes (1, 2)
    group_3_6 = np.sum([reclass_emerg1, reclass_scrub1, reclass_forest1, reclass_shore1], axis = 0)   # non water classes (3, 4, 5, 6)
    
    # Applying majority filter for non-water classes- This filter removes very small polygons and/or merges with the class which shares the longest boundary
    majority_result = rank.majority(group_3_6, footprint=disk(10))
    
    # Adding water classes to non-water classes
    water_classes = group1_2.copy()
    water_classes = np.where(water_classes == 0, 1, 0)
    all_classes = np.multiply(water_classes, majority_result)
    final_output = np.sum([all_classes, group1_2], axis = 0)
    
    # Resampling to 2.4 resolution
    
    # Define the target resolution (2.4 meters in this case)
    target_resolution = (2.4, 2.4)  # (x_resolution, y_resolution) in meters
    
    # Calculate the new dimensions based on the target resolution
    new_width = int(clipped_src.width * (clipped_src.res[0] / abs(target_resolution[0])))
    new_height = int(clipped_src.height * (clipped_src.res[1] / abs(target_resolution[1])))

    if new_width <= 0 or new_height <= 0:
        raise ValueError("Invalid target resolution. Check if it is compatible with the input image dimensions.")

    # Define the transform for the output image
    transform = from_origin(clipped_bounds.left, clipped_bounds.top, *target_resolution)

    # Create a profile for the output image
    output_profile = clipped_src.profile.copy()
    output_profile.update(
        width=new_width,
        height=new_height,
        transform=transform,
        dtype='uint8',  # Change the dtype as needed
        nodata=None  # Set nodata value as needed
    )

    # Create an output raster dataset
    memfile_io = io.BytesIO()
    print('mem', memfile_io)
    with rasterio.open(memfile_io, 'w', **output_profile) as dst:
        # Perform resampling and write to the output dataset
        reproject(
            source= final_output,
            destination=rasterio.band(dst, 1),
            src_transform=clipped_src.transform,
            src_crs=clipped_src.crs,
            dst_transform=transform,
            dst_crs=clipped_src.crs,  # Use the same CRS as the source
            resampling=Resampling.nearest  # Change the resampling method as needed
        )
    addm.addMeta_final(memfile_io, file, dst, aws_access_key_id,
                  aws_secret_access_key, s3_bucket_name)
        # Close the src object
    clipped_src.close()
    # Close the in-memory file
    memfile.close()
    
    
